# RSDI 2018 Capstone Project
## Harvard Forest Species Identification

### Spectral Indices Calculations Notebook
Christopher Kilner
christopher.kilner@duke.edu
July 13, 2018

*If in other sciences, we should arrive at certainty without doubt and thruth without error, it behooves us to place the foundations of knowledge in Mathematics*

In [2]:
#Install/Import necessary packages
import h5py
import numpy as np
import gdal
import os
import matplotlib.pyplot as plt
from math import floor
from copy import copy

In [1]:
#Define necessary functions
def h5refl2array(h5_filename):
    hdf5_file = h5py.File(h5_filename,'r')

    #Get the site name
    file_attrs_string = str(list(hdf5_file.items()))
    file_attrs_string_split = file_attrs_string.split("'")
    sitename = file_attrs_string_split[1]
    refl = hdf5_file[sitename]['Reflectance']
    reflArray = refl['Reflectance_Data']
    refl_shape = reflArray.shape
    wavelengths = refl['Metadata']['Spectral_Data']['Wavelength']
    #Create dictionary containing relevant metadata information
    metadata = {}
    metadata['shape'] = reflArray.shape
    metadata['mapInfo'] = refl['Metadata']['Coordinate_System']['Map_Info']
    #Extract no data value & set no data value to NaN\n",
    metadata['scaleFactor'] = float(reflArray.attrs['Scale_Factor'])
    metadata['noDataVal'] = float(reflArray.attrs['Data_Ignore_Value'])
    metadata['bad_band_window1'] = (refl.attrs['Band_Window_1_Nanometers'])
    metadata['bad_band_window2'] = (refl.attrs['Band_Window_2_Nanometers'])
    metadata['projection'] = refl['Metadata']['Coordinate_System']['Proj4'].value
    metadata['EPSG'] = int(refl['Metadata']['Coordinate_System']['EPSG Code'].value)
    mapInfo = refl['Metadata']['Coordinate_System']['Map_Info'].value

    mapInfo_string = str(mapInfo); #print('Map Info:',mapInfo_string)\n",
    mapInfo_split = mapInfo_string.split(",")
    #Extract the resolution & convert to floating decimal number
    metadata['res'] = {}
    metadata['res']['pixelWidth'] = mapInfo_split[5]
    metadata['res']['pixelHeight'] = mapInfo_split[6]
    #Extract the upper left-hand corner coordinates from mapInfo\n",
    xMin = float(mapInfo_split[3]) #convert from string to floating point number\n",
    yMax = float(mapInfo_split[4])
    #Calculate the xMax and yMin values from the dimensions\n",
    #xMax = left edge + (# of columns * resolution)\n",
    xMax = xMin + (refl_shape[1]*float(metadata['res']['pixelWidth'])) 
    #yMin = top edge - (# of rows * resolution)\n",
    yMin = yMax - (refl_shape[0]*float(metadata['res']['pixelHeight'])) 
    metadata['extent'] = (xMin,xMax,yMin,yMax),
    metadata['ext_dict'] = {}
    metadata['ext_dict']['xMin'] = xMin
    metadata['ext_dict']['xMax'] = xMax
    metadata['ext_dict']['yMin'] = yMin
    metadata['ext_dict']['yMax'] = yMax
    hdf5_file.close        
    return reflArray, metadata, wavelengths

#Raster to Array
def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array,epsg):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(epsg)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [ ]:
#Read in and confirm through exploration the Clean Hyperspectral Data
h5_directory = './data/Uncertainty/F07A/'
all_files = os.listdir(h5_directory)
h5_files = [i for i in all_files if i.endswith('.h5')]

In [ ]:
#Spectral Indice Definitions
wBlue = 470
wRed = 650
wGreen = Index[34]
wNIR = 860
wPRI1 = 531
wPRI2 = 570
wLignin2 = 1754
wLigNitro = 1680
wNitro1 = 1510
Gamma = 1

#For loop for multiple tiles
for f, file in enumerate(h5_files): 
    print('Working on ' + file)
    print('Counter (f): ' + str(f))
    
    reflArray, metadata, wavelengths = h5refl2array(h5_directory+file)
    
    Index_Blue = np.where(np.amin(np.abs(np.subtract(wavelengths,wBlue))) == (np.abs(np.subtract(wavelengths,wBlue))))
    Index_Red = np.where(np.amin(np.abs(np.subtract(wavelengths,wRed))) == (np.abs(np.subtract(wavelengths,wRed))))
    Index_Green = np.where(np.amin(np.abs(np.subtract(wavelengths,wGreen))) == (np.abs(np.subtract(wavelengths,wGreen))))
    Index_NIR = np.where(np.amin(np.abs(np.subtract(wavelengths,wNIR))) == (np.abs(np.subtract(wavelengths,wNIR))))
    Index_PRI1 = np.where(np.amin(np.abs(np.subtract(wavelengths,wPRI1))) == (np.abs(np.subtract(wavelengths,wPRI1))))
    Index_PRI2 = np.where(np.amin(np.abs(np.subtract(wavelengths,wPRI2))) == (np.abs(np.subtract(wavelengths,wPRI2))))
    Index_Lignin2 = np.where(np.amin(np.abs(np.subtract(wavelengths,wLignin2))) = (np.abs(np.subtract(wavelengths,wLignin2))))
    Index_LigninNitro = np.where(np.amin(np.abs(np.subtract(wavelengths,wLigninNitro))) == (np.abs(np.subtract(wavelengths,wLigninNitro))))
    Index_Nitro1 = np.where(np.amin(np.abs(np.subtract(wavelengths,wNitro1))) == (np.abs(np.subtract(wavelengths,wNitro1))))
    
    Blue = reflArray[:,:,Index_Blue]
    Red = reflArray[:,:,Index_Red]
    Green = reflArray[:,:,Index_Green]
    NIR = reflArray[:,:,Index_NIR]
    PRI1 = reflArray[:,:,Index_PRI1]
    PRI2 = reflArray[:,:,Index_PRI2]
    Lignin2 = reflArray[:,:,Index_Lignin2]
    LigNitro = reflArray[:,:,Index_LigNitro]
    Nitro1 = reflArray[:,:,Index_Nitro1]
    
    ENVI_Indices = np.zeros((reflArray.shape[0], reflArray.shape[1], len(7))
    
    #Spectral Indice Calculations
    ENVI_Indices[:,:,0] = (NIR - Red)/(NIR + Red)
    ENVI_Indices[:,:,1] = 2.5*((NIR-Red)/(NIR + (6*Red)-(7*Blue)+1))
    ENVI_Indices[:,:,2] = (NIR - (Red - Gamma(Blue - Red))/(NIR + (Red - Gamma(Blue - Red)))
    ENVI_Indices[:,:,3] = (PRI1 - PRI2)/(PRI1 + PRI2)
    ENVI_Indices[:,:,4] = (np.log(1/Lignin2)-np.log(1/LigNitro))/(np.log(1/Lignin2)+np.log(1/LigNitro))
    ENVI_Indices[:,:,5] = (np.log(1/Nitro)-np.log(1/LigNitro))/(np.log(1/Nitro)+np.log(1/LigNitro))
    ENVI_Indices[:,:,6] = (1.5*(NIR - Red))/(NIR + Red + 0.5)
    
    array2raster('ENVI_Index_'+filename_split[5]'.tif',(xMin,yMax),1,-1,np.array(ENVI_Indices,dtype=float),EPSG)